In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math

In [2]:
G = nx.karate_club_graph()

In [3]:
import Tr_centrality_measure
import Graph_Entropy_measure
import Selection_Algorithm
import Constraint_Coefficient
#N1=(PI/SUM(PI,TC,CC))
#N2=(TC/SUM(PI,TC,CC))
#N3=(CC/SUM(PI,TC,CC))
#TPN=N1*(PI_i)+N1*(TC_i)+N1*(CC_i)
i=0
def TPN(i):
    PI_i = Graph_Entropy_measure.PI_of(i)
    TC_i = Tr_centrality_measure.TC_of(i)
    CC_i = Constraint_Coefficient.CC_of(i)
    #CC_i=
    N1=PI_i/(PI_i+TC_i+CC_i)
    N2=TC_i/(PI_i+TC_i+CC_i)
    N3=CC_i/(PI_i+TC_i+CC_i)
    
    TPN_i = N1*(PI_i) + N2*(TC_i)+ N2*(CC_i)
    
    return TPN_i

In [4]:
#loop
def loop_TPN():
    i=1
    list_TPN=[]    
    for i in Selection_Algorithm.sel_subgraphs():          
        TPN_ = TPN(i)
        i+=1
        i_TPN = [i, TPN_]
        list_TPN.append(i_TPN)  
            
    return  list_TPN

In [5]:
def TPN_Ranking( val ):
      return val [1]

TPN_Rank = loop_TPN()
TPN_Rank.sort(key=lambda elem: elem[1])

#TPN_Rank

In [6]:
def sel_subgraphs():
    sel_subgraphs = []
    sel_subgraph = G.subgraph(Selection_Algorithm.nodes_in_triangle(G, 0))
    for i in sel_subgraph:                   
            row = i        
            sel_subgraphs.append(row)
            my_sel_subgraphs = tuple(sel_subgraphs)
    my_sel_subgraphs
    
sel_subgraphs()

In [8]:
plt.clf()
G.remove_node(sel_subgraphs.my_sel_subgraphs)
#G.remove_edge(Selection_Algorithm.subgraph_of(33).edges())
nx.draw(G)
plt.show()
plt.savefig("out.png")

AttributeError: 'NoneType' object has no attribute 'my_sel_subgraphs'

<Figure size 432x288 with 0 Axes>